### Merge tweets and stock price dataframes
Load all cleaned tweets, which are saved in separate files, load all stock prices and combine them in single dataframe grouped by date.

In [31]:
import os
import pandas as pd
# reset colwitdth options when running all cells 
pd.reset_option('display.max_colwidth')
pd.__version__

'0.25.3'

In [32]:
# read in all the tweets
tweets_dfs = []
directory = '../data/processed/less_strict/'
for subdir, dirs, files in os.walk(directory):
    for file in files:
        tweets_dfs.append(pd.read_json(directory+file))
        
tweets = pd.concat(tweets_dfs)

In [33]:
tweets

,timestamp,hashtags,text,username,likes,replies,retweets
6,2018-01-01 23:56:37,[],Busqué la recomendación de @jfcarpio sobre La ...,Musa de miope,2,0,1
8,2018-01-01 23:56:02,[],Pretty much every AV team other than Tesla is ...,Benedict Evans,1,2,0
14,2018-01-01 23:51:51,[],Tesla - named for Nikola Tesla - a household n...,Nicole Matejic,0,1,1
24,2018-01-01 23:56:37,[],Busqué la recomendación de @jfcarpio sobre La ...,Musa de miope,2,0,1
26,2018-01-01 23:56:02,[],Pretty much every AV team other than Tesla is ...,Benedict Evans,1,2,0
...,...,...,...,...,...,...,...
699181,2018-04-02 00:04:18,[],Lol you’re funny. Anyone who deigns speak ill ...,Andrew,0,2,0
699190,2018-04-02 00:03:35,[],Cuenta con mi mujer y con migo vamos desde Bil...,Pablo mr,2,1,0
699192,2018-04-02 00:03:25,[],My niece now has a Tesla pic.twitter.com/A9am1...,flower kind,4,3,0
699197,2018-04-02 00:03:00,[],Freud señalaba que la gracia del chiste es el ...,Rafael Skiadaressis,3,0,0


In [34]:
# uncomment lines below to generate a dataframe report
# CAUTION: TAKES ABOUT AN HOUR
# import pandas_profiling as profile
# profile = profile.ProfileReport(tweets, title='Daily Tweets Profiling Report', html={'style':{'full_width':True}})
# profile.to_file(output_file="merged_tweets_less_strict_df_report.html")

In [35]:
# the dataframe report has shown that there are some rows with duplicate tweet texts... remove these
tweets.drop_duplicates(subset='text', keep='first', inplace=True)
tweets.shape

(220019, 7)

In [36]:
# remove the time information as we only have stock price data per day
# tweets['timestamp'] = pd.DatetimeIndex(tweets['timestamp']).normalize()
tweets['timestamp'] = tweets['timestamp'].dt.date
tweets.head(2)

,timestamp,hashtags,text,username,likes,replies,retweets
6,2018-01-01,[],Busqué la recomendación de @jfcarpio sobre La ...,Musa de miope,2,0,1
8,2018-01-01,[],Pretty much every AV team other than Tesla is ...,Benedict Evans,1,2,0


### Load the stock market data and merge both dataframes

In [37]:
# load stock data
stocks = pd.read_json('../data/processed/stock/stocks_cleaned.json')
stocks.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 106 entries, 2018-01-02 to 2018-06-04
Data columns (total 3 columns):
Open       106 non-null float64
Close      106 non-null float64
PriceUp    106 non-null bool
dtypes: bool(1), float64(2)
memory usage: 2.6 KB


In [38]:
# compare with tweets df
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220019 entries, 6 to 699209
Data columns (total 7 columns):
timestamp    220019 non-null object
hashtags     220019 non-null object
text         220019 non-null object
username     220019 non-null object
likes        220019 non-null int64
replies      220019 non-null int64
retweets     220019 non-null int64
dtypes: int64(3), object(4)
memory usage: 13.4+ MB


In [39]:
# change tweets index to be the timestamp 
# to be able to merge it with the stock market df
tweets['timestamp'] = pd.to_datetime(tweets['timestamp'])
tweets.set_index('timestamp', inplace=True, drop=False)
tweets.sort_index(inplace=True)
tweets.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 220019 entries, 2018-01-01 to 2018-06-04
Data columns (total 7 columns):
timestamp    220019 non-null datetime64[ns]
hashtags     220019 non-null object
text         220019 non-null object
username     220019 non-null object
likes        220019 non-null int64
replies      220019 non-null int64
retweets     220019 non-null int64
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 13.4+ MB


In [40]:
# combine both datasets with an inner merge, which will remove days with missing stock prices
data = pd.merge(tweets, stocks, left_index=True, right_index=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 176520 entries, 2018-01-02 to 2018-06-04
Data columns (total 10 columns):
timestamp    176520 non-null datetime64[ns]
hashtags     176520 non-null object
text         176520 non-null object
username     176520 non-null object
likes        176520 non-null int64
replies      176520 non-null int64
retweets     176520 non-null int64
Open         176520 non-null float64
Close        176520 non-null float64
PriceUp      176520 non-null bool
dtypes: bool(1), datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 13.6+ MB


In [41]:
data.head(3)

,timestamp,hashtags,text,username,likes,replies,retweets,Open,Close,PriceUp
2018-01-02,2018-01-02,[],Tesla's scant disclosures leave Wall Street gu...,Automotive News,11,2,9,312.0,320.53,True
2018-01-02,2018-01-02,[],Elon Musk and Tesla said earlier this year tha...,Martin Baccardax,2,0,1,312.0,320.53,True
2018-01-02,2018-01-02,"[ManAndBeast, drivefreehttps]",An amazing accomplishment and a testament to b...,Tesla Model S60,7,0,2,312.0,320.53,True


### Group data by days 
... and investigate...

In [42]:
# group data by day
daily_data = data.groupby(data['timestamp'], as_index=False)
daily_data.first()

,timestamp,hashtags,text,username,likes,replies,retweets,Open,Close,PriceUp
0,2018-01-02,[],Tesla's scant disclosures leave Wall Street gu...,Automotive News,11,2,9,312.00,320.53,True
1,2018-01-03,[],Traditional truck maker Navigant claims it wil...,AukeHoekstra,44,6,25,321.00,317.25,False
2,2018-01-04,[],Tesla Falls After Musk Delays Model 3 Producti...,BRANDON Edwards,1,1,1,312.87,314.62,True
3,2018-01-05,[],Nearly 80% Of Electric Cars (Minus Tesla) Are ...,InsideEVs,8,3,3,316.62,316.58,False
4,2018-01-08,[],"En 1926, Nikola Tesla predijo el smartphone co...",Gizmodo en Español,84,1,58,316.00,336.41,True
...,...,...,...,...,...,...,...,...,...,...
98,2018-05-29,[],How is that awful? It’s the same advice I giv...,Gregg Peppel,31,1,0,278.51,283.76,True
99,2018-05-30,"[Autopilot, GuidaAutonoma, TeslaModel3, tuttotek]",Tesla Model 3: tour mondiale con schianto fina...,tuttoteK,0,0,1,283.29,291.72,True
100,2018-05-31,[],My first time in a @Tesla and my my what an aw...,Rakesh Nagaraj,10,2,1,287.21,284.73,False
101,2018-06-01,[],Blood spray detection would works. (Kidding......,Tom Robinson,2,0,0,285.86,291.82,True


In [48]:
# count tweets per day to see if they're ok'ish distributed
tweets_per_day = daily_data['text'].count()
tweets_per_day.describe()

,text
count,103.000000
mean,1713.786408
std,1220.137140
min,218.000000
25%,1251.000000
50%,1419.000000
75%,1821.500000
max,10320.000000


### Observations 
During merging, we've lost 3 of 106 days where stock data is available due to missing relevant tweets on that day. Looking at the mean and std of tweet numbers per day, the tweets most probably were skipped during mining.

In average, there are 1700 tweets per day with a minimum of 218 tweets. The standard deviation is quite high too, but since we're so far only looking at individual tweets, this is absolutely ok. Even when we go for averaging the tweets of a single day, it should still be fine.

Another important investigation is how many tweets we have for days with "PriceUp"==True and False?

In [70]:
price_groups = data.groupby(data['PriceUp'])
n_false, n_true = price_groups['text'].count()
print("Number of tweets in both groups: {} and {}".format(n_true, n_false))
print("Percentage of tweets with PriceUp == True: {:.3f}".format(n_true/(n_true+n_false)))

Number of tweets in both groups: 91160 and 85360
Percentage of tweets with PriceUp == True: 0.516


In [46]:
# check how to access all tweets of a single day
# get groups' names
daily_data.groups.keys()
groups = [name for name, _ in daily_data]
groups[0]

Timestamp('2018-01-02 00:00:00')

In [47]:
# get all tweets from the first day
first_day_data = daily_data.get_group(groups[0])
first_day_data.head()

,timestamp,hashtags,text,username,likes,replies,retweets,Open,Close,PriceUp
2018-01-02,2018-01-02,[],Tesla's scant disclosures leave Wall Street gu...,Automotive News,11,2,9,312.0,320.53,True
2018-01-02,2018-01-02,[],Elon Musk and Tesla said earlier this year tha...,Martin Baccardax,2,0,1,312.0,320.53,True
2018-01-02,2018-01-02,"[ManAndBeast, drivefreehttps]",An amazing accomplishment and a testament to b...,Tesla Model S60,7,0,2,312.0,320.53,True
2018-01-02,2018-01-02,[],Tesla's scant disclosures on the Model 3 leave...,Businessweek,2,0,2,312.0,320.53,True
2018-01-02,2018-01-02,[],Like Abraham Lincoln? Tesla? Edison? Traveling...,Science Traveler,0,0,3,312.0,320.53,True


### Generate Report and save the merged data

In [27]:
# save the full merged dataset
data.reset_index(inplace=True)
# rename index column to timestamp
data.rename(columns={'index':'timestamp'}, inplace=True)

data.to_json('processed_data/data_less_strict_merged.json')

data.head(3)

,timestamp,hashtags,text,username,likes,replies,retweets,Open,Close,PriceUp
0,2018-01-02,[],Tesla's scant disclosures leave Wall Street gu...,Automotive News,11,2,9,312.0,320.53,True
1,2018-01-02,[],Elon Musk and Tesla said earlier this year tha...,Martin Baccardax,2,0,1,312.0,320.53,True
2,2018-01-02,"[ManAndBeast, drivefreehttps]",An amazing accomplishment and a testament to b...,Tesla Model S60,7,0,2,312.0,320.53,True


In [14]:
# print the generated dataframe report
profile.to_notebook_iframe()